In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-21 14:23:22--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3,7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3,74K  --.-KB/s    in 0s      

2024-06-21 14:23:22 (71,6 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [4]:
import minsearch
import json

In [5]:
from openai import OpenAI
client = OpenAI()

In [13]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [14]:
def create_index(documents):
    index = minsearch.Index(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"],
    )
    index.fit(documents)
    return index

In [15]:
def search(index, query):
    boost = {'question': 3.0, 'section': 0.5}

    return index.search(
    query=query,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5,
)

In [16]:
def build_prompt(query, search_results):
    prompt_template= """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE

        QUESTION: {question}

        CONTEXT:
        {context}
    """
    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    return prompt_template.format(question=query, context=context).strip()


In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user', 'content': prompt}],
    )
    return response.choices[0].message.content

In [19]:
def rag(query):
    index = create_index(documents)
    results = search(index, query)
    prompt = build_prompt(query, results)
    return llm(prompt)

In [20]:
query = "The course already started. Can I enroll?"
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [24]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
index_name = "course_questions"
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [25]:
from tqdm.auto import tqdm

In [27]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 182.61it/s]


In [45]:
def elastic_search(client, query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit["_source"])
    return result_docs

In [47]:
query = "I have just joined the course, can I still take part?"
def rag(query):
    results = elastic_search(es_client, query)
    prompt = build_prompt(query, results)
    return llm(prompt)

rag(query)

"Yes, you can still join the course after the start date. Even if you haven't registered, you are still eligible to submit the homeworks. Just be mindful of the deadlines for turning in the final projects."